In [2]:
# Load packages
import requests 
import pandas as pd 
import time

from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By
import re
import time

In [3]:
base_url = 'https://www.consumeraffairs.com/food/uber-eats.html?page='
urls = [base_url + str(page) + '#scroll_to_reviews=true' for page in range(1, 5)]
print(urls)

['https://www.consumeraffairs.com/food/uber-eats.html?page=1#scroll_to_reviews=true', 'https://www.consumeraffairs.com/food/uber-eats.html?page=2#scroll_to_reviews=true', 'https://www.consumeraffairs.com/food/uber-eats.html?page=3#scroll_to_reviews=true', 'https://www.consumeraffairs.com/food/uber-eats.html?page=4#scroll_to_reviews=true']


In [5]:
resp = requests.get(urls[0])
content = BeautifulSoup(resp.content, 'lxml')
print(content)

<!DOCTYPE html>
<html class="no-js" lang="en"><head><script>window.ca_experiments = {};</script><!-- Google Tag Manager --><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.defer=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-WSBZRR');</script><!-- End Google Tag Manager --><meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="//media.consumeraffairs.com/static/manifest.9cfa8161eca4.json" rel="manifest"/><meta charset="utf-8"/><meta content="ConsumerAffairs" name="twitter:site" property="og:site_name"/><meta content="initial-scale=1,user-scalable=yes,width=device-width" name="viewport"/><meta content="www.consumeraffairs.com" name="twitter:domain"/><meta content="always" name="referrer"/><meta content="#000" name="theme-color"/><meta content=

In [6]:
def scrape_reviews(urls):

    service = Service(executable_path="C:\\Program Files (x86)\\chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    item_list = []
    for url in urls:
        driver.get(url)
        content = driver.find_elements(By.CLASS_NAME, 'js-rvw')

        for post in content:
            try:
                review = post.find_element(By.CLASS_NAME, 'rvw__top-text').text
                name = post.find_element(By.CLASS_NAME, 'rvw__inf-nm').text
                state = post.find_element(By.CLASS_NAME, 'rvw__inf-lctn').text

                # Use find_element to locate the desired element
                rating_element = driver.find_element(By.XPATH,'//meta[@itemprop="ratingValue"]')
                # Extract the rating value
                rating_value = rating_element.get_attribute('content')


                date_text = post.find_element(By.CLASS_NAME, 'rvw__rvd-dt').text

                # Use regex to extract the date in the desired format
                date_match = re.search(r"Reviewed (\w+)\. (\d{1,2}), (\d{4})", date_text)
                month, day, year = date_match.groups()

                # Convert the month abbreviation to the full month name
                month_name = pd.to_datetime(month, format='%b').strftime('%B')

                formatted_date = f"{month_name} {day}, {year}"

                data = {
                    "review": review,
                    "name": name,
                    "state": state,
                    "stars": rating_value,
                    "date": formatted_date,
                    'source':'Consumer Affairs',
                } 
            except:
                continue
            
            item_list.append(data)

    return pd.DataFrame(item_list)

In [7]:
df_reviews = scrape_reviews(urls)
df_reviews.to_csv("UberEats_consumeraffairs.csv", index= False)
df_reviews

,review,name,state,stars,date,source
0,My spouse accidentally placed an order on the ...,Aria,"San Diego, CA",1,"January 22, 2024",Consumer Affairs
1,"Seen a promotion, used the promotion before ch...",Aleta,"Columbus, OH",1,"January 22, 2024",Consumer Affairs
2,"Been on the phone for two hours, been transfer...",Erica,"Farmington, MI",1,"January 20, 2024",Consumer Affairs
3,I ordered with my wrong address. I then change...,le,"Zionsville, IN",1,"January 20, 2024",Consumer Affairs
4,I came upon a promotion on Uber Eats to purcha...,Ijeoma,"Winnipeg, MB",1,"January 19, 2024",Consumer Affairs
...,...,...,...,...,...,...
223,I received an email from Uber EATS on November...,Thelma,"Palm Bay, FL",1,"November 21, 2020",Consumer Affairs
224,I received a $25.00 coupon code via email. I’v...,Tarsha,"Fort Lauderdale, FL",1,"November 3, 2020",Consumer Affairs
225,My experience is similar to others who reviewe...,Jill,"Bethesda, MD",1,"November 1, 2020",Consumer Affairs
226,They emailed me a promo code for $30 off a fir...,Sandra,"Clover, SC",1,"October 27, 2020",Consumer Affairs
